<a href="https://colab.research.google.com/github/LUCASDNORONHA/Construindo-um-sistema-de-recomenda-o/blob/main/Construindo_um_sistema_de_recomenda%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Construindo um sistema de recomendação

Neste artigo iremos criar uma situação ficticia, onde seremos um engenheiro de ML que deve construir um sistema recomendação.

## Situação Fictícia:

Você é um engenheiro de Machine Learning em uma empresa de e-commerce. A empresa deseja aumentar as vendas recomendando produtos relevantes aos clientes. Eles já coletaram dados de transações anteriores, incluindo informações como:

- ID do cliente

- ID do produto

- Categoria do produto

- Data da compra

- Valor da compra

- Avaliação do cliente sobre o produto (de 1 a 5)


A tarefa é construir um sistema de recomendação para sugerir produtos aos clientes com base no histórico de compras e avaliações.

# Aplicando o Pensamento Analítico para uma Visão Estratégica no Desenvolvimento do Nosso Sistema de Recomendação

O pensamento analítico vai além de ser apenas uma metodologia; ele é uma forma de raciocinar e abordar problemas, independentemente da situação. Aplicar uma abordagem analítica de forma eficiente é crucial para alcançar os objetivos estabelecidos. Ter um pensamento analítico significa ser capaz de extrair informações valiosas tanto dos dados quanto da situação a ser resolvida, garantindo uma compreensão profunda e uma solução eficaz.

## Definição do Problema:
**Pergunta Principal:** Como podemos recomendar produtos relevantes aos clientes para aumentar as vendas?

**Subquestões:**
- Quais dados são mais relevantes para fazer recomendações?
- Qual algoritmo de recomendação seria mais adequado?

## Coleta de Dados:
- Revisar os dados disponíveis e identificar se há dados faltantes ou inconsistentes.
- Garantir que todos os dados necessários (ID do cliente, ID do produto, categoria, data da compra, valor e avaliação) estejam disponíveis.

## Análise Exploratória de Dados:
- Examinar a distribuição das avaliações dos produtos.
- Identificar padrões de compra, como produtos frequentemente comprados juntos.
- Verificar a correlação entre a categoria do produto e a avaliação média.

## Preparação dos Dados:
- Limpar os dados, removendo duplicatas e preenchendo valores ausentes.
- Normalizar os dados de avaliações para um formato consistente.
- Dividir os dados em conjuntos de treinamento e teste.

## Seleção do Modelo:
- Considerar diferentes algoritmos de recomendação, como Filtragem Colaborativa, Filtragem Baseada em Conteúdo e Modelos Híbridos.
- Testar os modelos utilizando validação cruzada e métricas como precisão, recall e F1-score.

## Treinamento e Avaliação do Modelo:
- Treinar o modelo escolhido com o conjunto de dados de treinamento.
- Avaliar o desempenho do modelo no conjunto de teste.
- Ajustar hiperparâmetros para melhorar o desempenho.

## Implementação e Monitoramento:
- Implementar o modelo de recomendação no sistema de e-commerce.
- Monitorar as recomendações em tempo real e coletar feedback dos clientes.
- Ajustar o modelo com base no feedback e em novos dados de transações.




# Geração dos Dados Fictícios

In [1]:
import numpy as np
import pandas as pd
import altair as alt
import plotly.express as px


from datetime import datetime, timedelta
import random

In [2]:
# Função para gerar datas aleatórias
def random_date(start, end):
  return start + timedelta(days=random.randint(0, int((end - start).days)))

In [3]:
# Gerar dados fictícios
num_records = 5344
user_ids = np.random.randint(1, 101, size=num_records)
item_ids = np.random.randint(1, 501, size=num_records)
categories = np.random.choice(['Eletrônicos', 'Roupas', 'Casa', 'Livros', 'Esportes', 'Beleza'], size=num_records)
purchase_dates = [random_date(datetime(2023, 1, 1), datetime(2024, 7, 1)) for _ in range(num_records)]
purchase_values = np.round(np.random.uniform(10, 500, size=num_records), 2)
ratings = np.random.randint(1, 6, size=num_records)

# Criar DataFrame
df = pd.DataFrame({
    'userID': user_ids,
    'itemID': item_ids,
    'category': categories,
    'purchaseDate': purchase_dates,
    'purchaseValue': purchase_values,
    'rating': ratings
})

# Introdução

Em nosso e-commerce, analisamos amostras de dados de compras recentes para entender melhor as preferências dos clientes e identificar padrões importantes que possam guiar nossas estratégias futuras.

# Exploração dos Dados

A tabela abaixo mostra uma amostra aleatória de 10 transações realizadas recentemente em nossa plataforma:

In [4]:
df.sample(10)

,userID,itemID,category,purchaseDate,purchaseValue,rating
444,74,150,Eletrônicos,2023-04-07,421.43,2
3571,50,406,Beleza,2024-05-03,177.51,2
4373,9,371,Esportes,2024-06-10,138.55,5
3292,25,460,Eletrônicos,2023-05-06,204.78,3
4499,28,5,Roupas,2023-09-19,105.77,2
4309,50,250,Eletrônicos,2023-03-24,439.83,5
42,22,355,Eletrônicos,2023-07-03,324.63,1
4504,42,465,Casa,2023-02-05,336.25,5
4827,1,176,Esportes,2023-06-01,178.79,1
1021,20,22,Esportes,2023-12-04,322.52,3


# Distribuição por Categorias de Produto

Para entender melhor a distribuição das categorias, criamos um gráfico de barras que mostra a contagem de transações por categoria:

Ao analisar os dados históricos de transações, descobrimos que temos um total de 6 categorias principais de produtos: Eletrônicos, Roupas, Casa, Livros, Esportes e Beleza.

O gráfico abaixo mostra a distribuição da contagem de produtos por categoria:

In [68]:
# Contar o número de ocorrências de cada categoria e calcular a amplitude
count_category = df['category'].value_counts().reset_index()
count_category.columns = ['category', 'Count']
amplitude = count_category['Count'].max() - count_category['Count'].min()

# Criar o gráfico de barras usando Altair
chart = alt.Chart(count_category).mark_bar().encode(
    x=alt.X('category:N', title='Categoria do Produto', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Count:Q', title='Contagem de Transações'),
    color=alt.Color('category:N', legend=alt.Legend(title=f'Amplitude: {amplitude}'))
).properties(
    title={'text': ['Distribuição de Contagem por Categoria de Produto'], 'subtitle': [f'O gráfico mostra a contagem de transações por categoria']},
    width=850,
    height=400
)

# Adicionar valores exatos de contagem sobre as barras
chart_text = chart.mark_text(
    align='center',
    baseline='bottom',
    dy=-5,
    size=14,
).encode(
    text='Count:Q'
)

# Combinar gráfico de barras com valores exatos e mostrar o gráfico final
final_chart = chart + chart_text
final_chart

alt.LayerChart(...)

## Insights Obtidos

- Roupas aparece como a categoria mais frequente em nossas transações, seguida por Casa e Beleza.

- Livros e Eletrônicos são as categorias menos populares, sugerindo uma oportunidade de crescimento ou estratégias específicas de marketing.

- Apesar disso, podemos observar que a diferença não é muito grande, tendo uma aplitude de 80 entre o maior e o menor número de transações por categoria.

# Análise Temporal das Vendas por Categoria

Vamos analisar a distribuição das vendas ao longo do tempo, com o objetivo de identificar quais categorias de produtos se destacaram em termos de volume de vendas.

In [66]:
# Converter a coluna purchaseDate para tipo datetime
df['purchaseDate'] = pd.to_datetime(df['purchaseDate'])

# Extrair mês e ano para facilitar a análise temporal
df['year_month'] = df['purchaseDate'].dt.to_period('M').astype(str)

# Agrupar por categoria e mês, somando o valor das compras
category_sales = df.groupby(['category', 'year_month'])['purchaseValue'].sum().reset_index()

# Criar o gráfico de linhas usando Plotly Express
fig = px.line(category_sales, x='year_month', y='purchaseValue', color='category',
              title='Vendas por Categoria ao Longo do Ano',
              labels={'year_month': 'Mês e Ano', 'purchaseValue': 'Valor das Compras', 'category': 'Categoria'},
              markers=True)
# Mostrar o gráfico
fig.show()

## Análise dos Resultados
1. Identificação de Tendências Sazonais
Ao analisar o gráfico, podemos identificar tendências sazonais nas vendas. Por exemplo, se observarmos um aumento nas vendas de Eletrônicos durante os meses de novembro e dezembro, isso pode indicar um aumento de compras durante a temporada de festas.

2. Comparação de Categorias
O gráfico também permite comparar o desempenho das diferentes categorias ao longo do ano. Podemos identificar quais categorias tiveram um desempenho consistente, quais tiveram picos de vendas específicos e quais categorias podem necessitar de maior atenção ou estratégias de marketing para melhorar suas vendas.

3. Insights para Estratégias de Marketing
Com base nas tendências observadas, a equipe de marketing pode planejar campanhas específicas para períodos de alta demanda ou identificar oportunidades para promover categorias menos vendidas durante períodos de baixa.